## Commands

In [ ]:
# titanv 1:
# screen -S generic-cache1
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/twinews-logs
# DOCKER_PORT=9963 nn -o nohup-generic-cache-$HOSTNAME-1.out ~/docker/keras/run-jupython.sh ~/notebooks/twinews/hjnotebooks/generic-cache.ipynb titanv
# observe ~/twinews-logs/nohup-generic-cache-$HOSTNAME-1.out

In [ ]:
# titanv 2:
# screen -S generic-cache2
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/twinews-logs
# DOCKER_PORT=9964 nn -o nohup-generic-cache-$HOSTNAME-2.out ~/docker/keras/run-jupython.sh ~/notebooks/twinews/hjnotebooks/generic-cache.ipynb titanv
# observe ~/twinews-logs/nohup-generic-cache-$HOSTNAME-2.out

In [ ]:
# oomstopper --no-tail generic-cache ; killbill generic-cache ; cd ~/twinews-logs ; jupython -o nohup-generic-cache-$HOSTNAME.out --venv st-venv ~/notebooks/twinews/hjnotebooks/generic-cache.ipynb

## Main params

In [ ]:
from twinews.models.genericutils import *
cacheFields = dictSelect(genericFields,\
{
    # 'bdert-ft',
    # 'bdert-base',
    # 'infersent',
    # 'sent2vec',
    # 'doc2vec',
    'bert',
    # 'stylo',
    # 'nmf',
    # 'tfidf',
    # 'word2vec',
})

## Imports

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = isNotebook # False, True, isNotebook

In [ ]:
import os
from systemtools.hayj import *
if labia():
    os.environ["CUDA_VISIBLE_DEVICES"] = ''
elif toCache(cacheFields, 'dbert'):
    os.environ["CUDA_VISIBLE_DEVICES"] = '1'
    import tensorflow as tf
    from tensorflow.keras import callbacks
    from transformers import \
    (
        DistilBertConfig,
        DistilBertTokenizer,
        TFDistilBertForSequenceClassification,
    )
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [ ]:
from systemtools.basics import *
from systemtools.printer import *
from systemtools.file import *
from systemtools.location import *
from datatools.jsonutils import *
from machinelearning.encoder import *
from newssource.asa2.utils import *
from machinelearning import tf2utils
from newssource.dbert.utils import *
from twinews.utils import *
from twinews.models.ranking import *

In [ ]:
import logging
import math

## Config

In [ ]:
logger = Logger('generic-cache-' + getHostname() + '.log')

In [ ]:
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
(user, password, host) = getMongoAuth(user='hayj')

In [ ]:
newsCollection = getNewsCollection(logger=logger)

In [ ]:
ids = newsCollection.distinct('_id')

In [ ]:
def getIdsChunks(n=200, verbose=True, maxIds=None, doShuffle=True):
    if TEST:
        n = 10
    if maxIds is not None:
        localIds = ids[:maxIds]
    else:
        localIds = ids
    if doShuffle:
        localIds = shuffle(localIds)
    if TEST:
        ch = chunks(localIds, n)[:4]
    else:
        ch = chunks(localIds, n)
    bp(ch, logger, verbose=verbose)
    return ch

In [ ]:
def basicGenFunct(containers, *args, field='detokText', yieldCache=True, cacheKey=None, logger=None, verbose=True, **kwargs):
    """
        This function yield the hash with the data (by field).
        If you gave a cacheKey, it will check if the row already exists in the cache.
        This function also yield the cache. So the yielded tuple is 3-sized.
    """
    if cacheKey is None:
        cache = None
        logWarning("You didn't give a cache key", logger, verbose=verbose)
    else:
        cache = getGenericCache(cacheKey, logger=logger, verbose=False)
    newsCollection = getNewsCollection(logger=logger, verbose=False)
    if not isinstance(containers[0], list):
        containers = [containers]
    for container in containers:
        for currentId in container:
            row = newsCollection.findOne({'_id': currentId}, projection={field})
            data = row[field]
            theHash = objectToHash(data)
            if cache is None:
                yield (theHash, data)
            elif theHash not in cache:
                if yieldCache:
                    yield (theHash, data, cache)
                else:
                    yield (theHash, data)

## DBert

In [ ]:
if toCache(cacheFields, 'dbert'):
    raise Exception("Need to re-implement this part (the cache key is dbert-ft)")
    # The best model is dbert-94bef_ep32-distilbert-mean
    modelNames = [] # ['94bef_ep32', None]
    if toCache(cacheFields, 'dbert-ft'):
        modelNames.append('94bef_ep32')
    if toCache(cacheFields, 'dbert-base'):
        modelNames.append(None)
    bp(modelNames, 5, logger)
    maxLength = 512
    batchSize = 16
    # layers = ['distilbert', 'pre_classifier', 'dropout', 'classifier'] if modelPath is not None else ['distilbert']
    layer = 'distilbert'

In [ ]:
if toCache(cacheFields, 'dbert'):
    raise Exception("Need to re-implement this part (the cache key is dbert-ft)")
    for modelName in modelNames:
        if modelName is None:
            modelPath = None
            modelName = 'base'
        else:
            if lri():
                modelPath = sortedGlob(nosaveDir() + "/dbert-tmp/" + modelName + "/epochs/ep*")[0]
            else:
                modelPath = sortedGlob(homeDir() + "/asa/dbert-tmp/" + modelName + "/epochs/ep*")[0]
        # We init the model:
        if modelPath is None:
            dbertConfig = DistilBertConfig.from_pretrained\
            (
                "distilbert-base-uncased",
                num_labels=1200,
                max_length=maxLength,
            )
            model = TFDistilBertForSequenceClassification.from_pretrained\
            (
                "distilbert-base-uncased",
                config=dbertConfig,
            )
        else:
            dbertConfig = DistilBertConfig.from_pretrained(modelPath + '/config.json')
            model = TFDistilBertForSequenceClassification.from_pretrained\
            (
                modelPath + '/tf_model.h5',
                config=dbertConfig,
            )
        log(modelName + " dbert model loaded.", logger)
        # We init the cache:
        cache = getGenericCache("dbert-" + modelName)
        # We find ids:
        ids = set()
        pbar = ProgressBar(len(newsCollection), printRatio=0.01, logger=logger, message="Finding not yet computed texts")
        for row in newsCollection.find({}, projection={'_id': True, 'detokText': True}):
            currentHash = objectToHash(row['detokText'])
            if currentHash not in cache:
                ids.add(row['_id'])
            pbar.tic()
        if TEST:
            ids = ids[:3]
        log(str(len(ids)) + " rows to process...", logger)
        # We init the Progress Bar:
        pbar = ProgressBar(len(ids), logger=logger, printRatio=0.0001, message="Generating vectors")
        # We infer all embeddings for each row in each file:
        for currentId in ids:
            row = newsCollection.findOne({'_id': currentId}, projection={'detokText': True})
            if dictContains(row, 'detokText'):
                text= row['detokText']
                currentHash = objectToHash(text)
                if currentHash not in cache:
                    encodedText = tf2utils.distilBertEncode\
                    (
                        text,
                        maxLength=maxLength,
                        multiSamplage=True,
                        preventTokenizerWarnings=True,
                    )
                    encodedBatches = chunks(encodedText, batchSize)
                    embeddings = []
                    for encodedBatch in encodedBatches:
                        outputs = tf2utils.getDistilBertRepresentations(model, np.array(encodedBatch), layer=layer)
                        for output in outputs:
                            embeddings.append(np.array(output))
                    embeddings = np.mean(embeddings, axis=0)
                    cache[currentHash] = embeddings
            pbar.tic()

## Stylo

Use `subprocessWrite = True` when you have few processing in sub-processes and `subprocessWrite = False` when sub processing is time consuming.

In [ ]:
model = 'stylo'
cacheKey = model
if toCache(cacheFields, model):
    subprocessWrite = True # False = 57 secs, True = 52 secs (for 50000 rows)
    idsChunks = getIdsChunks()
    cache = getGenericCache(cacheKey)
    field = genericFields[model]

In [ ]:
if toCache(cacheFields, model):
    def parseFunct(row, *args, **kwargs):
        (theHash, detokText, cache) = row
        styloVector = stylo(detokText, asNpArray=True)
        if subprocessWrite:
            cache[theHash] = styloVector
            return None
        else:
            return (theHash, styloVector)

In [ ]:
if toCache(cacheFields, model):
    mli = MLIterator\
    (
        idsChunks,
        basicGenFunct, genKwargs={'field': field, 'cacheKey': cacheKey},
        parseFunct=parseFunct,
        queuesMaxSize=1000, printRatio=0.01, parallelProcesses=cpuCount(),
        logger=logger,
    )

In [ ]:
if toCache(cacheFields, model):
    for row in mli:
        if not subprocessWrite:
            (theHash, styloVector) = row
            cache[theHash] = styloVector

## InferSent

Best is `V=1` and `method="mean"`

In [ ]:
model = 'infersent'
cacheKey = model
if TEST:
    cacheKey += "-test"
if toCache(cacheFields, model):
    idsChunks = getIdsChunks()
    cache = getGenericCache(cacheKey)
    field = genericFields[model]

In [ ]:
if toCache(cacheFields, model):
    import nltk
    nltk.download('punkt')
    gitClonesDir = tmpDir("git-clones")
    mkdir(gitClonesDir)
    infersentDir = gitClonesDir + "/infersent"
    bash("git clone https://github.com/facebookresearch/InferSent " + infersentDir)
    bash("touch " + infersentDir + "/__init__.py")
    sys.path.append(gitClonesDir)
    if isDocker():
        bash("pip install torch")
    from infersent.models import InferSent
    import torch

In [ ]:
if toCache(cacheFields, model):
    log("Loading InferSent model...", logger)
    V = 1
    MODEL_PATH = nosaveDir() + '/infersent/infersent%s.pkl' % V
    params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                    'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
    infersent = InferSent(params_model)
    infersent.load_state_dict(torch.load(MODEL_PATH))
    W2V_PATH = nosaveDir() + '/infersent/GloVe/glove.840B.300d.txt'
    infersent.set_w2v_path(W2V_PATH)
    log("Loading docs for vocab...", logger)
    detokDocs = [e["detokText"] for e in newsCollection.find({}, limit=200 if TEST else 0)]
    log("Building vocab...", logger)
    infersent.build_vocab(detokDocs, tokenize=True)
    detokDocs = None
    tt.tic("InferSent initialized.")

In [ ]:
if toCache(cacheFields, model):
    mli = MLIterator\
    (
        idsChunks,
        basicGenFunct, genKwargs={'field': field, 'cacheKey': cacheKey, 'yieldCache': False},
        queuesMaxSize=1000, printRatio=0.01, parallelProcesses=cpuCount(),
        logger=logger,
    )

In [ ]:
if toCache(cacheFields, model):
    for row in mli:
        (theHash, detokSentences) = row
        embedding = np.mean(infersent.encode(detokSentences, tokenize=True), axis=0)
        cache[theHash] = embedding

## Sent2Vec

In [ ]:
# pip install Cython ; git clone https://github.com/epfml/sent2vec.git ; cd ./sent2vec ; pip install .

In [ ]:
model = 'sent2vec'
cacheKey = model
if TEST:
    cacheKey += "-test"
if toCache(cacheFields, model):
    idsChunks = getIdsChunks(10000)
    cache = getGenericCache(cacheKey)
    field = genericFields[model]
    s2vModel = None

In [ ]:
if toCache(cacheFields, model):
    installSent2Vec()
    import sent2vec

In [ ]:
if toCache(cacheFields, model):
    def parseFunct(row, *args, logger=None, verbose=True, **kwargs):
        global s2vModel
        if s2vModel is None:
            log("Loading s2v model...", logger, verbose=verbose)
            s2vModel = sent2vec.Sent2vecModel()
            s2vModel.load_model(nosaveDir() + '/sent2vec/' + "wiki_unigrams.bin")
        (theHash, detokSentences) = row
        embedding = np.mean(s2vModel.embed_sentences(detokSentences), axis=0)
        return (theHash, embedding)

In [ ]:
if toCache(cacheFields, model):
    mli = MLIterator\
    (
        idsChunks,
        basicGenFunct, genKwargs={'field': field, 'cacheKey': cacheKey, 'yieldCache': False},
        parseFunct=parseFunct,
        queuesMaxSize=1000, printRatio=0.01, parallelProcesses=2 if TEST else cpuCount(),
        logger=logger,
    )

In [ ]:
if toCache(cacheFields, model):
    for row in mli:
        (theHash, embedding) = row
        cache[theHash] = embedding

## Word2Vec

In [ ]:
model = 'word2vec'
cacheKey = model
if TEST:
    cacheKey += "-test"
if toCache(cacheFields, model):
    idsChunks = getIdsChunks(10000)
    cache = getGenericCache(cacheKey)
    field = genericFields[model]
    wordEmbeddings = None
    wvPattern = "word2vec-googlenews"

In [ ]:
if toCache(cacheFields, model):
    def parseFunct(row, *args, logger=None, verbose=True, **kwargs):
        global wordEmbeddings
        if wordEmbeddings is None:
            log("Loading word2vec...", logger, verbose=verbose)
            dataDir = None
            if isHostname('tipi'):
                dataDir = "/special/hayj/Embeddings"
            emb = Embeddings(wvPattern, dataDir=dataDir, logger=logger, verbose=False)
            wordEmbeddings = emb.getVectors()
            assert len(wordEmbeddings) > 300
        (theHash, sentences) = row
        doc = flattenLists(sentences)
        if "The" not in wordEmbeddings:
            doc = [e.lower() for e in doc]
        embedding = tokensToEmbedding(doc, wordEmbeddings, operation='mean', removeDuplicates=True, verbose=False)
        return (theHash, embedding)

In [ ]:
if toCache(cacheFields, model):
    mli = MLIterator\
    (
        idsChunks,
        basicGenFunct, genKwargs={'field': field, 'cacheKey': cacheKey, 'yieldCache': False},
        parseFunct=parseFunct,
        queuesMaxSize=1000, printRatio=0.01, parallelProcesses=2 if TEST else cpuCount(),
        logger=logger,
    )

In [ ]:
if toCache(cacheFields, model):
    for row in mli:
        (theHash, embedding) = row
        cache[theHash] = embedding

## Doc2Vec

In [ ]:
def getD2v(path=nosaveDir() + "/d2v/d2vmodel-t-ds22.02g-s300-w3-n15-e15-lTrue-adFalse-7bb8a", loadModel=True):
    d2vConfig = None
    d2vModel = None
    if isFile(path + "/config.json"):
        d2vConfig = fromJsonFile(path + "/config.json")
    if loadModel:
        d2vModel = Doc2Vec.load(sortedGlob(path + "/*model*.d2v")[0])
    return (d2vModel, d2vConfig)

In [ ]:
model = 'doc2vec'
cacheKey = model
if TEST:
    cacheKey += "-test"
if toCache(cacheFields, model):
    from gensim.models.doc2vec import Doc2Vec
    idsChunks = getIdsChunks(10000)
    cache = getGenericCache(cacheKey)
    field = genericFields[model]
    d2vModel = None

In [ ]:
if toCache(cacheFields, model):
    def parseFunct(row, *args, logger=None, verbose=True, **kwargs):
        global d2vModel
        if d2vModel is None:
            log("Loading doc2vec...", logger)
            (d2vModel, d2vConfig) = getD2v()
        (theHash, detokSentences) = row
        doc = flattenLists(detokSentences)
        embedding = d2vTokenssToEmbeddings([doc], d2vModel, doLower=True, verbose=False)
        return (theHash, embedding)

In [ ]:
if toCache(cacheFields, model):
    mli = MLIterator\
    (
        idsChunks,
        basicGenFunct, genKwargs={'field': field, 'cacheKey': cacheKey, 'yieldCache': False},
        parseFunct=parseFunct,
        queuesMaxSize=1000, printRatio=0.01, parallelProcesses=1 if TEST else 4,
        logger=logger,
    )

In [ ]:
if toCache(cacheFields, model):
    for row in mli:
        (theHash, embedding) = row
        cache[theHash] = embedding

## USent

## BERT

In [ ]:
model = 'bert'
cacheKey = model
if TEST:
    cacheKey += "-test"
if toCache(cacheFields, model):
    bash("pip install bert-serving-client==1.10.0")
    from bert_serving.client import BertClient
    idsChunks = getIdsChunks()
    cache = getGenericCache(cacheKey)
    field = genericFields[model]
    bc = None # BertClient()

In [ ]:
if toCache(cacheFields, model):
    def parseFunct(row, *args, logger=None, verbose=True, **kwargs):
        global bc
        if bc is None:
            bc = BertClient(ip='titanv.lri.fr', check_length=False)
        (theHash, detokSentences, cache) = row
        bertBatch = bc.encode(detokSentences)
        bertBatchMean = np.mean(bertBatch, axis=0)
        cache[theHash] = bertBatchMean
        return None

In [ ]:
if toCache(cacheFields, model):
    mli = MLIterator\
    (
        idsChunks,
        basicGenFunct, genKwargs={'field': field, 'cacheKey': cacheKey, 'yieldCache': True},
        parseFunct=parseFunct,
        queuesMaxSize=1000, printRatio=0.01, parallelProcesses=4,
        logger=logger,
    )

In [ ]:
if toCache(cacheFields, model):
    for row in mli:
        pass

## End

In [ ]:
tt.toc()